<a href="https://colab.research.google.com/github/Kimeteor309/Financial-big-data-analysis/blob/main/%E9%87%91%E8%9E%8D%E5%A4%A7%E6%95%B8%E6%93%9A%E5%88%86%E6%9E%90_week12%E4%BD%9C%E6%A5%AD_%E7%94%9F%E6%88%90%E5%BC%8FAI%E5%88%86%E6%9E%90%E8%82%A1%E7%A5%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# 安裝yahoo finance套件
!pip install yfinance

In [3]:
pip install yahoo-fin

In [11]:
import requests
import pandas as pd
import yfinance as yf
import google.generativeai as genai

# 初始化 Gemini API
def initialize_gemini(api_key):
    """
    配置 Gemini API 金鑰。
    :param api_key: API 金鑰
    """
    genai.configure(api_key=api_key)

# 從 Yahoo Finance 獲取股票資料
def fetch_stock_data(stock_symbol, period="1y", interval="1d"):
    try:
        data = yf.download(stock_symbol, period=period, interval=interval)
        if data.empty:
            raise ValueError("無法獲取股票數據，請檢查代碼是否正確。")
        return data
    except Exception as e:
        raise Exception(f"錯誤: {str(e)}")

# 整合基本面分析
def fundamental_analysis(stock_symbol):
    stock = yf.Ticker(stock_symbol)
    try:
        info = stock.info
        analysis = {
            "市值": info.get("marketCap"),
            "本益比": info.get("trailingPE"),
            "每股盈餘 (EPS)": info.get("trailingEps"),
            "股息收益率": info.get("dividendYield"),
        }
        return analysis
    except Exception as e:
        return {"錯誤": str(e)}

# 整合技術面分析
def technical_analysis(data):
    # 提取收盤價
    close_prices = data['Close']

    # 計算均線
    data['MA_50'] = close_prices.rolling(window=50).mean()
    data['MA_200'] = close_prices.rolling(window=200).mean()

    # 獲取最新的數據
    latest_price = close_prices.iloc[-1] if not close_prices.empty else None
    moving_average_50 = data['MA_50'].iloc[-1] if not data['MA_50'].empty else None
    moving_average_200 = data['MA_200'].iloc[-1] if not data['MA_200'].empty else None

    # 確保值為單一浮點數而非 Series
    latest_price = float(latest_price)
    moving_average_50 = float(moving_average_50)
    moving_average_200 = float(moving_average_200)

    # 判斷交易建議
    if latest_price > moving_average_50 > moving_average_200:
        recommendation = "買進"
    else:
        recommendation = "賣出"


    return {
        "最新價格": latest_price,
        "50日移動平均線": moving_average_50,
        "200日移動平均線": moving_average_200,
        "建議操作": recommendation,
    }

# 新聞與情感分析
from yahoo_fin import news

def fetch_latest_news(stock_symbol):
    """
    從 Yahoo Finance 獲取指定股票的最新新聞。
    :param stock_symbol: 股票代碼 (如 "2330.TW")
    :return: 新聞標題列表
    """
    stock_news = news.get_yf_rss(stock_symbol)

    # 遍歷每條新聞
    cleaned_news = []
    for item in stock_news:
        # 確保每條新聞包含需要的字段
        title = item.get("title", "No Title")
        summary = item.get("summary", "No Summary")
        link = item.get("link", "No Link")
        pub_date = item.get("published", "Unknown Date")

        # 將每條新聞封裝到字典中
        cleaned_news.append({
            "title": title,
            "summary": summary,
            "link": link,
            "pub_date": pub_date
        })

    # 只保留最新的10筆新聞
    return cleaned_news[:10]

def news_analysis(stock_symbol):
    """
    使用 Gemini 模型分析新聞及情感。
    :param stock_symbol: 股票代碼
    :return: 新聞情感分析結果
    """
    news_items = fetch_latest_news(stock_symbol)

    analysis_results = []
    model = genai.GenerativeModel("gemini-1.5-flash")

    for news in news_items:
        prompt = (
            f"以下是股票代碼 {stock_symbol} 的一則新聞：\n\n"
            f"標題: {news['title']}\n"
            f"摘要: {news['summary']}\n"
            f"發布時間: {news['pub_date']}\n"
            f"連結: {news['link']}\n\n"
            "請進行情感分析，判斷是否正面、中立或負面，並總結其對該股票的可能影響。"
        )

        response = model.generate_content(prompt, generation_config=genai.types.GenerationConfig(
                max_output_tokens=1600,  # 限制回應的最大 Token 數為 1600
                temperature=1.0,  # 調整回應的隨機性（1.0 為適中的隨機性，範圍[0.0, 2.0]，值越高越有創意，值越低個準確）
            ))
        analysis_results.append({
            "title": news["title"],
            "summary": news["summary"],
            "link": news["link"],
            "pub_date": news["pub_date"],
            "analysis": response.text.strip()
        })

    return analysis_results

# 生成專業報告
def generate_report(stock_symbol):
    print(f"分析股票代碼: {stock_symbol}\n")

    # 基本面與技術面分析
    stock_data = fetch_stock_data(stock_symbol)
    fundamental = fundamental_analysis(stock_symbol)
    technical = technical_analysis(stock_data)

    print("基本面分析:")
    for key, value in fundamental.items():
        print(f"{key}: {value}")

    print("\n技術面分析:")
    for key, value in technical.items():
        print(f"{key}: {value}")

    # 使用 LLM 進行新聞與投資建議分析
    print("\n新聞分析:")
    results = news_analysis(stock_symbol)
    for result in results:
        print(f"新聞標題: {result['title']}")
        print(f"摘要: {result['summary']}")
        print(f"發布時間: {result['pub_date']}")
        print(f"連結: {result['link']}")
        print(f"情感分析: {result['analysis']}\n")

    print("\n專業投資建議:")
    prompt = (
        f"請根據以下基本面與技術面分析數據，以及新聞情感分析結果，"
        f"為股票代碼 {stock_symbol} 提供投資建議。\n\n"
        f"基本面分析: {fundamental}\n\n技術面分析: {technical}\n\n新聞與情感分析: {results}"
    )

    # 使用 Gemini 模型生成投資建議
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt, generation_config=genai.types.GenerationConfig(
                max_output_tokens=1600,  # 限制回應的最大 Token 數為 1600
                temperature=1.0,  # 調整回應的隨機性（1.0 為適中的隨機性，範圍[0.0, 2.0]，值越高越有創意，值越低個準確）
            ))
    report = response.text.strip()
    print(report)

In [12]:
if __name__ == "__main__":
    # 替換為你的 API 金鑰
    API_KEY = "替換為你的 API 金鑰"

    # 初始化 Gemini
    initialize_gemini(API_KEY)

    # 股票代碼
    stock_symbol = "2330.TW"

    # 生成專業報告
    generate_report(stock_symbol)

[*********************100%***********************]  1 of 1 completed

分析股票代碼: 2330.TW

基本面分析:
市值: 25828968628224
本益比: 24.592592
每股盈餘 (EPS): 40.5
股息收益率: 0.0159

技術面分析:
最新價格: 996.0
50日移動平均線: 1025.74
200日移動平均線: 897.91
建議操作: 賣出

新聞分析:



<ipython-input-11-45acb8aa1829>:54: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  latest_price = float(latest_price)


新聞標題: Taiwan Semiconductor Manufacturing (TSM): Begins Equipment Installation at Kaohsiung 2nm Fab, Trial Run Set for 2025
摘要: We recently compiled a list of the 15 AI Stocks That are Making Headlines. In this article, we are going to take a look at where Taiwan Semiconductor Manufacturing Company Limited (NYSE:TSM) stands against the other AI stocks. Perplexity, the California-based AI startup that is challenging the dominance of Google in the Internet Search market, is […]
發布時間: Thu, 28 Nov 2024 12:48:44 +0000
連結: https://finance.yahoo.com/news/taiwan-semiconductor-manufacturing-tsm-begins-124844411.html?.tsrc=rss
情感分析: 這則新聞標題和摘要呈現**正面**的情感。

**理由：**

* **標題:** 明確指出台積電(TSM)在高雄開始安裝2奈米晶圓廠設備，並預計2025年試產。這代表著台積電在先進製程技術上的持續領先地位，對投資者來說是積極的訊號。

* **摘要:** 雖然摘要大部分內容與台積電無直接關係，提到了其他AI股票，但新聞標題已經建立了正面基調。  摘要並未提及任何對台積電不利的訊息。


**對股票的可能影響：**

這則新聞很可能對2330.TW (台積電在台灣交易所的股票代碼)產生**正面影響**。  投資者通常會對先進製程技術的進展給予積極回應，因為這代表著未來的營收增長潛力。  預計2025年試產的消息，雖然還未真正量產，但已經開始逐步建立市場對台積電未來發展的信心，可能導致股價上漲或維持強勢。


**需要注意的點：**